In [17]:
class Gvozdomet:
    def init(self):
        self.img_path = None


    def connect_model(self):
        self.model = torch.hub.load('ultralytics/yolov5', 'custom', path='train_num_yolov5_weights.pt', force_reload=True)
        self.model.conf = 0.5 #нижняя планка вероятности для предсказания



    def get_number(self, pth):

        # self.model = torch.hub.load('ultralytics/yolov5', 'custom', path='/content/drive/MyDrive/num_weights/train_num_yolov5_weights.pt', force_reload=True)
        # self.model.conf = 0.5 #нижняя планка вероятности для предсказания

        img = Image.open(pth)
        img = img.resize((640, 640))
        self.results = self.model(img)

        return self.detect_train_number(pth)


    def detect_train_number(self, pth):

        reader = easyocr.Reader(['en'], gpu=False)

        img = cv2.resize(cv2.imread(pth), (640, 640))

        if self.results.xyxy[0].shape[0] > 0:
            mas = self.results.xyxy[0][0].cpu().numpy().tolist()[:-2]
            for i in range(len(mas)):
                mas[i] = int(round(mas[i], 0))


            img_rec = img[mas[1]:mas[3], mas[0]:mas[2]]


            img_rec = cv2.cvtColor(img_rec, cv2.COLOR_BGR2GRAY)


            detections = reader.readtext(img_rec, detail=0)
            arr = [pth]
            if len(detections) > 0:
                itog_nm = ""
                num = detections[0]
                for i in num:
                    if i in "0123456789":
                        itog_nm += i
                    elif i in "\|/":
                        itog_nm += "1"
                if len(itog_nm) > 0:
                    arr.append(1) #добавили 1 в type
                    arr.append(int(itog_nm)) # добавили номер


                    if len(itog_nm) == 8:


                        mnog = "21212121"
                        sm = ""
                        for i in range(len(itog_nm) - 1):
                            sm += str(int(itog_nm[i]) * int(mnog[i]))

                        itog_sm = 0
                        for i in sm:
                            itog_sm += int(i)


                        if 10 - (itog_sm % 10) == int(itog_nm[-1]):
                            arr.append(1) #добавили is_correct 1 если прошло
                        else:
                            arr.append(0)#добавили is_correct 0 если не прошло

                    else:
                        arr.append(0) #добавили is_correct 0 если длина != 8
                else:
                    arr.append(0)
                    arr.append(94012576)
                    arr.append(0)


        else: # заливаем форму при отсутствии
            arr.append(0)
            arr.append(94012576)
            arr.append(0)

        return arr

In [ ]:
!pip install easyocr

In [3]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 16003, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 16003 (delta 21), reused 20 (delta 13), pack-reused 15967
Receiving objects: 100% (16003/16003), 14.60 MiB | 3.96 MiB/s, done.
Resolving deltas: 100% (10987/10987), done.
/Users/vlyrdv/Desktop/python/hahaton_sochi/yolov5
Note: you may need to restart the kernel to use updated packages.


In [5]:
%matplotlib notebook
%matplotlib inline

In [ ]:
!pip install easyocr

In [7]:
from glob import glob
import easyocr
import torch
import utils
import cv2
from sklearn.metrics import mean_squared_error as mse
from PIL import Image
import random
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import numpy as np
display = utils.notebook_init()

YOLOv5 🚀 v7.0-226-gdd9e338 Python-3.11.4 torch-2.0.1 CPU


Setup complete ✅ (8 CPUs, 16.0 GB RAM, 233.6/460.4 GB disk)


In [8]:
%cd ..

/Users/vlyrdv/Desktop/python/hahaton_sochi


In [20]:
obj = Gvozdomet()

obj.connect_model()


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 v7.0-226-gdd9e338 Python-3.10.12 torch-2.0.1+cu118 CPU

Fusing layers... 
Model summary: 212 layers, 20852934 parameters, 0 gradients, 47.9 GFLOPs
Adding AutoShape... 


In [21]:
print(obj.get_number("/content/42343657.jpg"))

['/content/42343657.jpg', 1, 42343657, 1]


In [22]:
from flask import Flask, request, jsonify
import time


# Создание Flask-приложения
app = Flask(__name__)


@app.route('/')
def start():
    return "Привет дорогой друг"



# Определение API-маршрута
@app.route('/pred/<img_name>')
def predict(img_name):
    
    obj = Gvozdomet()
    obj.connect_model()
    # Получение изображения из тела запроса
    image_path = f"images/{img_name}.jpg"

    # Подача обработанного изображения на вход модели
    predictions = obj.get_number(image_path)

    # Возвращение результатов в формате JSON
    return {predictions[0] : {
        "time": time.time(),
        "type": predictions[1],
        "number": predictions[2],
        "is_correct": predictions[-1]
    }}


# Запуск сервера API
if __name__ == '__main__':
    app.run(host='127.0.0.1', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /Users/vlyrdv/.cache/torch/hub/master.zip
YOLOv5 🚀 v7.0-226-gdd9e338 Python-3.11.4 torch-2.0.1 CPU

Fusing layers... 
Model summary: 212 layers, 20852934 parameters, 0 gradients, 47.9 GFLOPs
Adding AutoShape... 
Using CPU. Note: This module is much faster with a GPU.
127.0.0.1 - - [14/Oct/2023 20:40:36] "GET /pred/42343657 HTTP/1.1" 200 -
